In [1]:
import nltk
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from music21 import *
import os

In [2]:
os.listdir('midi_file')

['LiveForever.mid',
 'MorningGlory.mid',
 'StandByMe.mid',
 'StopCryingYourHeartOut.mid',
 'SuperSonic.mid',
 'Whatever.mid',
 'Wonderwall.mid']

In [3]:
word_tokenize('ter ter ter')

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\terjr/nltk_data'
    - 'c:\\Users\\terjr\\AppData\\Local\\Programs\\Python\\Python39\\nltk_data'
    - 'c:\\Users\\terjr\\AppData\\Local\\Programs\\Python\\Python39\\share\\nltk_data'
    - 'c:\\Users\\terjr\\AppData\\Local\\Programs\\Python\\Python39\\lib\\nltk_data'
    - 'C:\\Users\\terjr\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [100]:
# from music21 import *

# # Load the MIDI file using music21
# midi_file = converter.parse('StandByMe.mid')

# # Get a list of notes from the flattened stream
# notes_list = list(midi_file.flat.notes)

# # Create a new stream to hold the notes
# reconstructed_stream = stream.Stream()

# # Add notes to the new stream
# for note in notes_list:
#     reconstructed_stream.append(note)

# # Create a MIDI file from the reconstructed stream
# output_midi_file = midi.translate.streamToMidiFile(reconstructed_stream)

# # Save the output MIDI file
# output_midi_file_path = 'output.mid'
# output_midi_file.open(output_midi_file_path, 'wb')
# output_midi_file.write()
# output_midi_file.close()

# print("MIDI file converted and saved as", output_midi_file_path)

In [101]:
from music21 import *

# Load the MIDI file using music21
midi_file = converter.parse('Wonderwall.mid')

# Get a list of notes from the flattened stream
# to convert to note with rest using list(midi_file.flatten().notesAndRests)
notes_list = list(midi_file.flat.notes)

# Create a new stream to hold the notes
reconstructed_stream = stream.Stream()

# Add notes to the new stream
for note in notes_list:
    reconstructed_stream.append(note)

# Create a MIDI file from the reconstructed stream
output_midi_file = midi.translate.streamToMidiFile(reconstructed_stream)

# Save the output MIDI file
output_midi_file_path = 'wdwall_renew.mid'
output_midi_file.open(output_midi_file_path, 'wb')
output_midi_file.write()
output_midi_file.close()

print("MIDI file converted and saved as", output_midi_file_path)

MIDI file converted and saved as wdwall_renew.mid


In [5]:
# Sample text
text = "This is a sample sentence for n-gram generation."

# Tokenize the text into words
words = word_tokenize('text')

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\terjr/nltk_data'
    - 'c:\\Users\\terjr\\AppData\\Local\\Programs\\Python\\Python39\\nltk_data'
    - 'c:\\Users\\terjr\\AppData\\Local\\Programs\\Python\\Python39\\share\\nltk_data'
    - 'c:\\Users\\terjr\\AppData\\Local\\Programs\\Python\\Python39\\lib\\nltk_data'
    - 'C:\\Users\\terjr\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [6]:
import nltk
from nltk.util import ngrams
 
samplText='this is a very good book to study'
NGRAMS=ngrams(sequence=nltk.word_tokenize(samplText), n=3)
for grams in NGRAMS:
    print(grams)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\terjr/nltk_data'
    - 'c:\\Users\\terjr\\AppData\\Local\\Programs\\Python\\Python39\\nltk_data'
    - 'c:\\Users\\terjr\\AppData\\Local\\Programs\\Python\\Python39\\share\\nltk_data'
    - 'c:\\Users\\terjr\\AppData\\Local\\Programs\\Python\\Python39\\lib\\nltk_data'
    - 'C:\\Users\\terjr\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************
